# SharePoint Columns – Arbeits‑Notebook (fokussiert)

> Zweck: Dieses Notebook erklärt **nur** das Modul **`graphfw.domains.sharepoint.lists.columns`** (
> Datei: `columns.py`), zeigt dessen wichtigste Methode & Parameter, demonstriert **Reloading via
> `graphfw.core.reloader.reload_df`** (nach Dateiänderungen) und liefert ausführliche sowie kurze
> Beispiel‑Snippets – inkl. gezielter **Spaltenauswahl** und **Filterung auf einen Content Type**.


## 1) Modulübersicht & API (nur `columns.py`)

**Modul**: `graphfw.domains.sharepoint.lists.columns` (Datei: `columns.py`)

**Kernfunktion**

```python
list_df(gc, *, site_url, list_title, mode="standard|extended|item", page_size=200, top=None, timeout=60, columns=None, expand=False, item_content_type=None, log=None) -> (pd.DataFrame, dict)
```

**Aufgabe**
- Ermittelt das **Spaltenschema** einer SharePoint‑Liste über Microsoft Graph. Liefert einen DataFrame mit **deterministischer Spaltenreihenfolge** sowie ein `info`‑Dict (Diagnostics).

**Wichtige Features**
- **Modi**: `standard` (nur /columns), `extended` (Items + `$expand=fields`), `item` (pro Content Type ein Beispiel‑Item).
- **GUID‑Sicherheit**: Spalte `GUID` wird sichergestellt (ggf. synthetisiert).
- **Spaltenauswahl**: `columns=[...]` matcht `internalName` **oder** `displayName` (tolerant ggü. SharePoint‑Encodings `_xNNNN_`, Groß-/Kleinschreibung etc.).
- **Diagnostics**: `info` enthält Quellen, Parameter, Warnungen, Auflösungsreport.

**Rückgabe‑Schema**
- `df`‑Spalten (fixe Reihenfolge, sofern nicht explizit gewählt): `['internalName','displayName','type','required','readOnly','hidden','indexed','enforceUnique','details','source','itemContentTypes']`
- `info`: u. a. `{ 'mode', 'list_path', 'expand', 'params', 'sources', 'warnings', 'resolution', 'module_version' }`


## 2) Reloading des Moduls (nur `columns.py`)

**Wann verwenden?**
- Immer dann, wenn Sie **Code in `columns.py` geändert** haben und das Notebook bereits läuft. Nutzen Sie dafür das **neue Hilfsmodul** `graphfw.core.reloader` und dessen Funktion **`reload_df([...])`**.

**Was passiert?**
- Es wird ein DataFrame mit den Spalten **Module | Version before | Version after** ausgegeben. Optional auch Dateipfade.


In [ ]:
# Setup & Reload‑Utility (nur columns)
import sys, os
proj_root = os.path.abspath(os.path.join(os.getcwd()))
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

from graphfw.core.reloader import reload_df

modules_to_reload = [
    "graphfw.domains.sharepoint.lists.columns",
]

reload_overview_df, reload_info = reload_df(modules_to_reload)
reload_overview_df  # Anzeige: Module | Version before | Version after


## 3) Beispiele & Erklärungen (ausführlich)

> In allen Beispielen wird der **HTTP‑Zugriff ausschließlich über `GraphClient`** vorgenommen. Erstellen Sie ihn mit einem `TokenProvider`.


In [ ]:
# Auth & Client – Platzhalter (mit echten Werten ersetzen)
# from graphfw.core.auth import TokenProvider
# from graphfw.core.http import GraphClient
# tp = TokenProvider.from_json("./secrets/graph.json")
# gc = GraphClient(tp)
gc = ...  # GraphClient Instanz einsetzen
site_url = "https://contoso.sharepoint.com/sites/TeamA"
list_title = "Aufgaben"


### 3.1 Spaltenschema (alle Felder, Modus `item`)
**Parameter**: `site_url`, `list_title`, `mode`, `columns`, `expand`, `item_content_type`, `page_size`, `top`, `timeout`.


In [ ]:
from graphfw.domains.sharepoint.lists import columns

# df, info = columns.list_df(
#     gc,
#     site_url=site_url,
#     list_title=list_title,
#     mode="item",              # 'standard' | 'extended' | 'item'
#     page_size=200,
#     columns=None,              # alle
#     expand=False,              # nur für mode='standard' relevant
#     item_content_type=None,    # optional: ID oder Name
# )
# display(df.head())
# info


## 4) Kurzfassung (Minimal‑Snippets)


In [ ]:
# Reload nur für columns
from graphfw.core.reloader import reload_df
reload_df(["graphfw.domains.sharepoint.lists.columns"])


In [ ]:
# Minimal – alle Spalten
# from graphfw.domains.sharepoint.lists import columns
# df, info = columns.list_df(gc, site_url=site_url, list_title=list_title)


## 5) Explizit bestimmte Spalten anfordern
**Ziel**: Nur die gewünschten Spalten (in genau der gewünschten Reihenfolge) zurückgeben.


In [ ]:
# Gewünschte Spalten definieren
wanted = ["Title", "GUID", "Created", "Modified", "Status"]

# df_sel, info_sel = columns.list_df(
#     gc,
#     site_url=site_url,
#     list_title=list_title,
#     mode="extended",      # oder 'standard'/'item'
#     columns=wanted,
# )
# display(df_sel)
# info_sel.get("warnings")  # z. B. nicht gefundene Spalten


## 6) Filterung auf einen bestimmten Content Type
**Ziel**: Spaltenerkennung und Beispiel‑Item nur für **einen** Content Type durchführen.


In [ ]:
# Nur CT "Dokument" berücksichtigen (oder CT-ID wie '0x0101...')
# df_ct, info_ct = columns.list_df(
#     gc,
#     site_url=site_url,
#     list_title=list_title,
#     mode="item",
#     item_content_type="Dokument",
# )
# display(df_ct.head())
# info_ct.get("warnings")


## 7) Aktuelle Modulversion anzeigen & nach Änderungen neu laden (nur columns)


In [ ]:
# Versionen zeigen & neu laden
reload_overview_df, reload_info = reload_df(["graphfw.domains.sharepoint.lists.columns"])
reload_overview_df


## 8) Troubleshooting & Hinweise
- **Import‑Fehler**: Projekt‑Root dem `sys.path` hinzufügen; Paketstruktur `graphfw/...` prüfen.
- **Berechtigungen**: Für echte Graph‑Calls benötigt Ihre App passende Scopes (Application/Delegated).
- **Keine Secrets loggen**: `TokenProvider`/Konfiguration dürfen keine Secrets ausgeben.
- **Deterministische Spalten**: Bei `columns=[...]` folgt die Reihenfolge genau Ihrer Liste; fehlende Spalten in `info['warnings']`.
